In [5]:
from collections import defaultdict
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import spacy, stanza


In [2]:
nlp_ja = spacy.load('ja_core_news_sm')  # Japonais
nlp_zh = spacy.load('zh_core_web_sm')   # Chinois
nlp_ar = stanza.Pipeline(lang='ar')    # Arabe
nlp_default = spacy.load('en_core_web_sm')  # Modèle par défaut pour les autres langues


def segment_sentences(text: str, language: str) -> list:
    """
    Segmenter un texte en phrases.

    Parameters:
    text (str): Le texte à segmenter.
    language (str): La langue du texte.

    Returns:
    List[str]: Une liste de phrases.
    """
    if language == 'ar':
        doc = nlp_ar(text)
        return [sentence.text.strip() for sentence in doc.sentences]
    else:
        if language == 'ja':
            doc = nlp_ja(text)
        elif language == 'zh':
            doc = nlp_zh(text)
        else:   
            doc = nlp_default(text)
        return [sent.text.strip() for sent in doc.sents]


def create_csv(folder: str, folder_csv: str) -> None:
    """
    Crée un fichier CSV à partir des fichiers texte dans le dossier donné.
    Chaque ligne du fichier CSV contient une paire de phrases et la langue de ces phrases.
    Les fichiers texte doivent être nommés selon le format suivant : "langue_nom.txt".

    Parameters:
    folder (str): Le chemin du dossier contenant les fichiers texte.
    folder_csv (str): Le chemin du dossier où le fichier CSV sera enregistré.

    Returns:
    None
    """
    data_by_language = defaultdict(list)

    for file in os.listdir(folder):
        lang = file.split("_")[0]
        with open(os.path.join(folder, file), "r", encoding="utf-8") as f:
            text = f.read()
        if text:
            # Utiliser sent_tokenize pour diviser le texte en phrases
            sentences = segment_sentences(text, lang)
            # Ajouter des paires de phrases
            if len(sentences) > 1:
                for i in range(0, len(sentences) - 1, 2):  # S'arrêter à l'avant-dernière phrase pour éviter un index hors limites
                    data_by_language[lang].append(' '.join(sentences[i:i+2]))
            else:
                data_by_language[lang].append(' '.join(sentences))
    
    balanced_data = []
    for lang, pairs in data_by_language.items():
        # Créer des tuples (langue, paire de phrases) pour le DataFrame
        balanced_data.extend((lang, pair) for pair in pairs)


    df_balanced = pd.DataFrame(balanced_data, columns=["Label", "Text"])
    os.makedirs(folder_csv, exist_ok=True)
    df_balanced.to_csv(os.path.join(folder_csv, "data.csv"), index=False, encoding="utf-8")


2024-04-29 21:16:09 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-04-29 21:16:09 INFO: Downloaded file to /Users/perrine/stanza_resources/resources.json
2024-04-29 21:16:10 INFO: Loading these models for language: ar (Arabic):
| Processor | Package       |
-----------------------------
| tokenize  | padt          |
| mwt       | padt          |
| pos       | padt_charlm   |
| lemma     | padt_nocharlm |
| depparse  | padt_charlm   |
| ner       | aqmar_charlm  |

2024-04-29 21:16:10 INFO: Using device: cpu
2024-04-29 21:16:10 INFO: Loading: tokenize
2024-04-29 21:16:10 INFO: Loading: mwt
2024-04-29 21:16:10 INFO: Loading: pos
2024-04-29 21:16:10 INFO: Loading: lemma
2024-04-29 21:16:10 INFO: Loading: depparse
2024-04-29 21:16:10 INFO: Loading: ner
2024-04-29 21:16:11 INFO: Done loading processors!


Je définis une fonction appelée `create_csv` qui prend deux paramètres : `folder` et `folder_csv`. `folder` est le chemin du dossier contenant les fichiers textuels que je veux analyser, et `folder_csv` est le chemin du dossier où je souhaite enregistrer le fichier CSV final.

Voici comment je m'organise pour traiter les données :

1. **Extraction et segmentation des phrases** :
   - Je crée un dictionnaire `data_by_language` en utilisant `defaultdict(list)`. Cela permet de collecter des paires de phrases, organisées par langue.
   - Je parcours tous les fichiers dans le dossier spécifié avec `os.listdir(folder)`. Pour chaque fichier, je détermine la langue en extrayant la première partie du nom du fichier (avant le premier '_'). Cela suppose que chaque fichier est nommé selon le format "langue_nom.txt".
   - J'ouvre chaque fichier en lecture, en utilisant son chemin complet et l'encodage UTF-8. Je lis son contenu et utilise `sent_tokenize` pour diviser le texte en phrases. Je parcours ces phrases par paires et les stocke dans le dictionnaire sous la clé correspondant à la langue du fichier.

2. **Préparation des données pour le CSV** :
   - Après avoir traité tous les fichiers, je parcours le dictionnaire `data_by_language`. Pour chaque langue, je prépare des tuples composés de la langue et des paires de phrases pour créer un ensemble de données structuré.

3. **Création du DataFrame et enregistrement en CSV** :
   - Je convertis l'ensemble de données en un DataFrame Pandas nommé `df_balanced`, avec les colonnes "Label" pour la langue et "Text" pour le texte des paires de phrases.
   - Je vérifie si le dossier `folder_csv` existe et le crée s'il n'existe pas avec `os.makedirs`.
   - Je sauvegarde le DataFrame dans un fichier CSV sous le nom "data.csv" dans le dossier `folder_csv`, en m'assurant de ne pas inclure l'index du DataFrame dans le fichier CSV et d'utiliser l'encodage UTF-8.

Cette méthode `create_csv` permet de structurer efficacement des paires de phrases par langue dans un fichier CSV, facilitant les analyses linguistiques ou autres traitements ultérieurs.


In [3]:
def create_train_test_csv(csv: str, folder_csv: str) -> None:
    """
    Divise un fichier CSV en un ensemble d'entraînement et un ensemble de test.
    Les ensembles d'entraînement et de test contiennent 80% et 20% des données respectivement.

    Parameters:
    csv (str): Le chemin du fichier CSV.
    folder_csv (str): Le chemin du dossier où les fichiers CSV seront enregistrés.

    Returns:
    None
    """
    df_train, df_test = train_test_split(csv, test_size=0.2, random_state=42)
    df_train.to_csv(os.path.join(folder_csv, "data_train.csv"), index=False, encoding="utf-8")
    df_test.to_csv(os.path.join(folder_csv, "data_test.csv"), index=False, encoding="utf-8")


Je définis une fonction appelée `create_train_test_csv` qui prend deux paramètres : `csv` et `folder_csv`. Le premier, `csv`, est un DataFrame qui contient les données, tandis que `folder_csv` est le chemin du dossier où je souhaite enregistrer les fichiers CSV résultants pour les ensembles d'entraînement et de test.

Voici comment je procède pour diviser les données et les enregistrer :

1. **Division des données en ensembles d'entraînement et de test** :
   - J'utilise la fonction `train_test_split` de la bibliothèque scikit-learn pour diviser le DataFrame `csv` en deux sous-ensembles. Je spécifie `test_size=0.2` pour allouer 20% des données au jeu de test, tandis que les 80% restants constitueront le jeu d'entraînement. L'argument `random_state=42` est utilisé pour garantir la reproductibilité du découpage des données.

2. **Enregistrement des ensembles d'entraînement et de test en CSV** :
   - Une fois les données divisées, je sauvegarde l'ensemble d'entraînement dans un fichier CSV nommé "data_train.csv" et l'ensemble de test dans un fichier "data_test.csv". Ces fichiers sont enregistrés dans le dossier spécifié par `folder_csv`.
   - Pour chaque sauvegarde, je spécifie `index=False` pour ne pas inclure l'index du DataFrame dans le fichier CSV et `encoding='utf-8'` pour assurer que le fichier est encodé correctement, ce qui est important pour le traitement ultérieur des données, surtout si elles contiennent des caractères non ASCII.

Cette fonction `create_train_test_csv` permet de préparer efficacement les données pour des phases ultérieures de modélisation en machine learning, en s'assurant que les données sont correctement réparties et enregistrées de manière à faciliter leur accès et leur utilisation.

In [4]:
def main():
    folder_csv = "../../../data/csv/"
    folder_txt_files = "../../../data/clean/"
    create_csv(folder_txt_files, folder_csv)
    df_balanced = pd.read_csv(os.path.join(folder_csv, "data.csv"), encoding="utf-8")
    create_train_test_csv(df_balanced, folder_csv)

main()